<a href="https://colab.research.google.com/github/melihkurtaran/MachineLearning/blob/main/UnsupervisedLearning/UnsupervisedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Unsupervised Learning Project**

In [ ]:
#Import Libraries
import numpy as np
import pandas as pd